# Logging en Python

> Objectif : comprendre l'utilité du logging, apprendre à configurer et utiliser le module `logging` dans des scripts et des projets data. Ce notebook contient des explications courtes, des exemples exécutables et des bonnes pratiques.




## 1) Pourquoi utiliser le logging ?

- `print()` est utile en dev rapide, mais **le logging** est conçu pour :
  - gérer différents niveaux d'urgence (DEBUG / INFO / WARNING / ERROR / CRITICAL),
  - router les messages vers plusieurs destinations (console, fichier, socket),
  - structurer et formater les messages (timestamp, module, contexte),
  - activer/désactiver la verbosité sans changer le code source,
  - mieux s'intégrer aux outils de monitoring et aux environnements de production (containers, systèmes de log centralisés).

**Cas d'usage data** : suivre l'avancement d'un pipeline, logger les tailles de dataset, timestamps d'étapes, erreurs d'ETL, métriques légères, etc.


## 2) Concepts clés

- **Logger** : objet qui émet des événements (nommé, ex: `my_app.module`).
- **Handler** : destination des logs (console, fichier, rotating file, HTTP...).
- **Formatter** : format du message enregistré.
- **Level** : seuil de sévérité (DEBUG < INFO < WARNING < ERROR < CRITICAL).
- **Filter** : filtre fin par message/logger.


## 3) Exemple minimal — commencer simplement

- Utiliser `getLogger(__name__)` dans les modules pour obtenir un logger nommé.
- Configurer un logger de base dans le `if __name__ == "__main__"`.

In [4]:
import logging
# configuration basique — utile en dev
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
)

logger = logging.getLogger(__name__)

logger.debug("Message debug — utile pour dev")
logger.info("Début du script")
logger.warning("Un avertissement")
logger.error("Une erreur simulée")

2025-08-14 13:57:42,680 [INFO] __main__ - Début du script
2025-08-14 13:57:42,682 [WARNING] __main__ - Un avertissement
2025-08-14 13:57:42,683 [ERROR] __main__ - Une erreur simulée


## 5) Handlers — écrire dans un fichier ou utiliser rotation

Exemple : console + fichier avec rotation (fichier limité à 5Mo, garder 3 backups).


In [5]:
import logging
from logging.handlers import RotatingFileHandler
from pathlib import Path

log_dir = Path("logs")
log_dir.mkdir(exist_ok=True)

logger = logging.getLogger("my_data_pipeline")
logger.setLevel(logging.DEBUG)  # logger en DEBUG localement

# StreamHandler pour la console
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(
    logging.Formatter("%(asctime)s [%(levelname)s] %(name)s:%(lineno)d - %(message)s")
)

# RotatingFileHandler pour fichier
fh = RotatingFileHandler(log_dir / "pipeline.log", maxBytes=5_000_000, backupCount=3)
fh.setLevel(logging.DEBUG)
fh.setFormatter(
    logging.Formatter("%(asctime)s [%(levelname)s] %(name)s - %(message)s")
)

# Attacher handlers (attention à ne pas attacher plusieurs fois dans REPL)
if not logger.handlers:
    logger.addHandler(sh)
    logger.addHandler(fh)

logger.info("Logger configuré avec console + rotation")
logger.debug("Debug écrit dans le fichier mais pas en console")

2025-08-14 13:59:00,862 [INFO] my_data_pipeline:30 - Logger configuré avec console + rotation
2025-08-14 13:59:00,862 [INFO] my_data_pipeline - Logger configuré avec console + rotation
2025-08-14 13:59:00,868 [DEBUG] my_data_pipeline - Debug écrit dans le fichier mais pas en console


## 6) Exemple avancé : dictConfig (configuration déclarative)

`logging.config.dictConfig` permet de configurer l'ensemble via un dictionnaire (pratique pour charger depuis YAML/TOML).

In [9]:
import logging.config

LOG_CONFIG = {
    "version": 1,
    "disable_existing_loggers": False,
    "formatters": {
        "standard": {
            "format": "%(asctime)s [%(levelname)s] %(name)s - %(message)s"
        }
    },
    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "formatter": "standard",
            "level": "INFO",
        },
        "file": {
            "class": "logging.handlers.RotatingFileHandler",
            "formatter": "standard",
            "level": "DEBUG",
            "filename": "logs/dictconfig_pipeline.log",
            "maxBytes": 1_000_000,
            "backupCount": 2,
        },
    },
    "loggers": {
        "": {"handlers": ["console", "file"], "level": "DEBUG"},
    },
}

logging.config.dictConfig(LOG_CONFIG)
logger = logging.getLogger(__name__)
logger.info("Logger via dictConfig actif")


2025-08-14 14:01:42,903 [INFO] __main__ - Logger via dictConfig actif
